In [1]:
from backtester.core.context import Context
from backtester.strategies.straddle1 import StrategyStraddle1
from backtester.utils.mktdata import createVolSurface

In [ ]:
filaname = r'C:\Users\Administrator\PycharmProjects\backtester\data\spx_vols.txt'
mktdata = createVolSurface(filaname)
context = Context(mktdata, balance=10000)

strategy = StrategyStraddle1(context)
strategy.run()

C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
mktdata.query("'2015-10-1'< Date < '2015-10-7'")

,Date,Spot,DaysToMaturity,Volatility,Maturity,Stock
0,2015-10-02,1951.36,1,0.184735,2015-10-03,.SPY
1,2015-10-02,1951.36,2,0.181799,2015-10-04,.SPY
2,2015-10-02,1951.36,3,0.179894,2015-10-05,.SPY
3,2015-10-02,1951.36,4,0.178889,2015-10-06,.SPY
4,2015-10-02,1951.36,5,0.178650,2015-10-07,.SPY
5,2015-10-02,1951.36,6,0.179047,2015-10-08,.SPY
6,2015-10-02,1951.36,7,0.179946,2015-10-09,.SPY
7,2015-10-02,1951.36,8,0.181217,2015-10-10,.SPY
8,2015-10-02,1951.36,9,0.182726,2015-10-11,.SPY
9,2015-10-02,1951.36,10,0.184342,2015-10-12,.SPY


In [15]:
dfPositionsHistOptions = pd.concat(strategy.optionsPortfolio.dfPositionsHist)
dfPositionsHistFutures = pd.concat(strategy.stockPortfolio.dfPositionsHist)
dfNotional = strategy.dfNotional
dfExpiryHist = strategy.optionsPortfolio.dfExpiryHist
dfTradeHistOptions = strategy.optionsPortfolio.dfTradeHist
dfTradeHistStock = strategy.stockPortfolio.dfTradeHist

df3 = pd.DataFrame(strategy.notionalDf, columns=['Date', 'Balance', 'OptionsNotional'])
df4 = pd.merge(df1, df2, how='outer', on=['Date'])
df5 = pd.merge(df4, df3, how='outer', on=['Date'])
#df5.info()
#df5
df5.to_csv('c:/temp/results2.csv')
#from IPython.display import display, HTML
#display(HTML(df.to_html()))

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
